In [ ]:
%matplotlib                    # Pyplot 그래프 새창에서 열기 옵션

In [ ]:
%matplotlib inline             # Pyplot 그래프 새창에서 열기 '취소' 옵션

In [ ]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm,metrics
import joblib

## 데이터 불러오기

In [2]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label'  , sep=",", header=None).T.squeeze())
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label', sep=",", header=None).T.squeeze())
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label', sep = ",", header = None).T.squeeze())

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(288, 30)
(72, 30)
(288,)
(72,)
(360, 30)


## SVM(Support Vector Machine) 기반 모델링 및 성능평가

In [3]:
Add    = 0
Divide = 0

for i in range(Fold):
    c1 = 'Training_CurrentFold = Training_Fold%d'%(i+1)
    exec(c1)
    c2 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c2)    
    
    svmModel_CurrentFold = svm.SVC(kernel='linear')
    svmModel_CurrentFold.fit(Training_CurrentFold , TrainingFold_Label)
    Predicted = np.array(svmModel_CurrentFold.predict(Validation_CurrentFold))
    
    c3 = 'svmscore_Fold%d = metrics.accuracy_score(ValidationFold_Label , Predicted)'%(i+1)
    exec(c3)
        
    Add += metrics.accuracy_score(ValidationFold_Label , Predicted)
    Divide += 1
    
Avg_accuracy = Add/Divide

print('[Result of K-fold Cross Validation] \n')
print(' Fold 1: {:.2f}% \n Fold 2: {:.2f}% \n Fold 3: {:.2f}% \n Fold 4: {:.2f}% \n Fold 5: {:.2f}%'.
        format(svmscore_Fold1*100, svmscore_Fold2*100, svmscore_Fold3*100, svmscore_Fold4*100, svmscore_Fold5*100))
print('\n Average accuracy: {:.2f}%'.format(Avg_accuracy*100))

[Result of K-fold Cross Validation] 

 Fold 1: 97.22% 
 Fold 2: 100.00% 
 Fold 3: 100.00% 
 Fold 4: 98.61% 
 Fold 5: 100.00%

 Average accuracy: 99.17%


## SVM model 모델 생성(전체 데이터) 및 저장

Pickle 패키지(라이브러리)를 이용한 머신러닝 모델 저장

In [4]:
SVM_model = svm.SVC(kernel='linear')
SVM_model.fit(Training_All, Training_All_Label)

joblib.dump(SVM_model, './MLmodels/SVM_model.plk')

['./MLmodels/SVM_model.plk']

## SVM model 불러와서 진단

저장된 KNN 모델 불러오기

In [5]:
SVM_model_Final = joblib.load('./MLmodels/SVM_model.plk')

Fold별 검증데이터에 대한 진단 정확도 평가 (Fold별 학습데이터만 학습한 모델에 비해 성능향상 확인)

In [6]:
Add    = 0
Divide = 0

for i in range(Fold):
    c1 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c1)    
    
    Predicted = SVM_model_Final.predict(Validation_CurrentFold)
    
    c2 = 'svmscore_Fold%d = metrics.accuracy_score(ValidationFold_Label , Predicted)'%(i+1)
    exec(c2)
        
    Add += metrics.accuracy_score(ValidationFold_Label , Predicted)
    Divide += 1
    
Avg_accuracy = Add/Divide

print('[Performance of SVM model] \n')
print(' Fold 1: {:.2f}% \n Fold 2: {:.2f}% \n Fold 3: {:.2f}% \n Fold 4: {:.2f}% \n Fold 5: {:.2f}%'.
        format(svmscore_Fold1*100, svmscore_Fold2*100, svmscore_Fold3*100, svmscore_Fold4*100, svmscore_Fold5*100))
print('\n Average accuracy: {:.2f}%'.format(Avg_accuracy*100))

[Performance of SVM model] 

 Fold 1: 97.22% 
 Fold 2: 100.00% 
 Fold 3: 100.00% 
 Fold 4: 100.00% 
 Fold 5: 100.00%

 Average accuracy: 99.44%
